# Data Preprocessing

In [184]:
# load library
import numpy as np
import pandas as pd
import zipfile
import urllib.request
import io
import os
import warnings
import sys
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_classif
from scipy.stats import entropy
warnings.filterwarnings('ignore')

In [185]:
# load raw data
url = "https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/ehr_preprocessed_seq_by_day_cat_embedding.pkl.zip"
with urllib.request.urlopen(url) as response:
    with zipfile.ZipFile(io.BytesIO(response.read())) as zip_file:
        with zip_file.open("ehr_preprocessed_seq_by_day_cat_embedding.pkl", "r") as file:
            EHR = pd.read_pickle(file)
train = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/train.csv")
valid = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/valid.csv")
test = pd.read_csv("https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/raw_data/test.csv")
print(train.shape)
print(test.shape)
print(valid.shape)
print(len(EHR['feature_cols']))

(55941, 13)
(17933, 12)
(13598, 13)
171


In [186]:
# drop image-related columns
train = train.drop(train.columns[[6,7,8,11]], axis=1)
valid = valid.drop(valid.columns[[6,7,8,11]], axis=1)
test = test.drop(test.columns[[6,7,8,11]], axis=1)
train.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days
0,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410828,90151.343,0
1,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410831,130627.031,0
2,10869829_25238191,10869829,25238191,2141-08-20 12:41:00,2141-09-01 13:22:00,NaN,21410821,80228.937,0
3,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880405,200636.062,1
4,17910612_22301530,17910612,22301530,2188-03-04 19:49:00,2188-04-19 00:00:00,2188-04-19 00:00:00,21880321,114135.046,1


In [187]:
# extract last-day observations
train['StudyDate'] = pd.to_datetime(train['StudyDate'], format='%Y%m%d')
train_latest_date_idxs = train.groupby('id')['StudyDate'].idxmax()
train_masked = train.loc[train_latest_date_idxs]

valid['StudyDate'] = pd.to_datetime(valid['StudyDate'], format='%Y%m%d')
valid_latest_date_idxs = valid.groupby('id')['StudyDate'].idxmax()
valid_masked = valid.loc[valid_latest_date_idxs]

test['StudyDate'] = pd.to_datetime(test['StudyDate'], format='%Y%m%d')
test_latest_date_idxs = test.groupby('id')['StudyDate'].idxmax()
test_masked = test.loc[test_latest_date_idxs]

print(train_masked.shape)
print(valid_masked.shape)
print(test_masked.shape)

(9271, 9)
(2325, 9)
(2936, 8)


In [188]:
# drop rows where deathtime is not empty for train, valid and test
#train_masked = train_masked[train_masked['deathtime'].isnull()]
#valid_masked = valid_masked[valid_masked['deathtime'].isnull()]
#test_masked = test_masked[test_masked['deathtime'].isnull()]

In [189]:
# merge train and valid
train_valid = pd.concat([train, valid])
train_valid_masked = pd.concat([train_masked, valid_masked])
print(train_valid.shape)
print(train_valid_masked.shape)

(69539, 9)
(11596, 9)


In [190]:
# add two new predictors (length of stay + previous number of admissions)
# length of stay
test_masked['dischtime'] = pd.to_datetime(test_masked['dischtime'])
test_masked['admittime'] = pd.to_datetime(test_masked['admittime'])
test_masked['stay_len'] = (test_masked['dischtime'] - test_masked['admittime']).dt.days
valid_masked['dischtime'] = pd.to_datetime(valid_masked['dischtime'])
valid_masked['admittime'] = pd.to_datetime(valid_masked['admittime'])
valid_masked['stay_len'] = (valid_masked['dischtime'] - valid_masked['admittime']).dt.days
train_valid_masked['dischtime'] = pd.to_datetime(train_valid_masked['dischtime'])
train_valid_masked['admittime'] = pd.to_datetime(train_valid_masked['admittime'])
train_valid_masked['stay_len'] = (train_valid_masked['dischtime'] - train_valid_masked['admittime']).dt.days
train_masked['dischtime'] = pd.to_datetime(train_masked['dischtime'])
train_masked['admittime'] = pd.to_datetime(train_masked['admittime'])
train_masked['stay_len'] = (train_masked['dischtime'] - train_masked['admittime']).dt.days

# previous number of admissions
test_masked = test_masked.sort_values('admittime')
test_masked['prev_admits'] = test_masked.groupby('subject_id').cumcount()
valid_masked = valid_masked.sort_values('admittime')
valid_masked['prev_admits'] = valid_masked.groupby('subject_id').cumcount()
train_valid_masked = train_valid_masked.sort_values('admittime')
train_valid_masked['prev_admits'] = train_valid_masked.groupby('subject_id').cumcount()
train_masked = train_masked.sort_values('admittime')
train_masked['prev_admits'] = train_masked.groupby('subject_id').cumcount()

train_masked.head(5)

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days,stay_len,prev_admits
41856,17195991_23542772,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaN,2110-01-16,90654.546,0,6,0
24332,13721591_20342223,13721591,20342223,2110-02-09 18:13:00,2110-02-22 20:51:00,NaN,2110-02-19,41948.468,0,13,0
20119,19170541_22178312,19170541,22178312,2110-02-28 21:48:00,2110-03-12 17:47:00,NaN,2110-03-11,81842.812,0,11,0
51567,15554295_27705504,15554295,27705504,2110-03-09 03:54:00,2110-05-18 11:34:00,NaN,2110-05-04,60653.312,0,70,0
29152,17643026_29919541,17643026,29919541,2110-03-25 11:15:00,2110-03-29 17:17:00,NaN,2110-03-28,140521.453,0,4,0


In [191]:
# extract feature column names
cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
cat_cols.extend(icd_cols)
num_cols = list(set(EHR['feature_cols']) - set(cat_cols)) 
#num_cols.extend(['stay_len', 'prev_admits']) 
print(cat_cols)
print(num_cols)

['gender', 'ethnicity', 'Creatinine Blood', 'Sodium Blood', 'pO2 Blood', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Lactate Blood', 'Anion Gap Blood', 'Eosinophils Joint Fluid', 'Hemoglobin Blood', 'Chloride Blood', 'Eosinophils Ascites', 'pH Urine', 'Calcium, Total Blood', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Eosinophils Blood', 'Lymphocytes Blood', 'Basophils Joint Fluid', 'Hematocrit Blood', 'Potassium Blood', 'H Blood', 'Monocytes Blood', 'Eosinophils Pleural', 'Troponin T Blood', 'Neutrophils Blood', 'Bicarbonate Blood', 'Basophils Blood', 'Glucose Blood', 'Basophils Ascites', 'pH Blood', 'Platelet Count Blood', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'Lymphocytes Pleural', 'pCO2 Blood', 'Y90-Y99', 'G30-G32', 'O85-O92', 'C60-C63', 'F40-F48', 'M80-M85', 'R00-R09', 'J90-J94', 'A00-A09', 'E00-E07', 'F01-F09', 'F30-F39', 'H30-H36', 'D60-D64', 'N00-N08', 'F60-F69', 'I80-I89', 'I95-I99', 'N30-N39', 'K55-K64', 'F50-F59',

In [193]:
# train_valid set
train_valid_merged_admits_all = []
train_valid_merged_admits_latest = []
train_valid_merged_admits_mean_mode = []
train_valid_merged_admits_entropy_std = []

for _, admit in train_valid_masked.iterrows():
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    temp_dfs = []
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = admit.append(feat_arr)
        merged['obs_id'] = i + 1  
        temp_dfs.append(merged.to_frame().transpose())
        train_valid_merged_admits_all.append(merged)
        if i == num_feat_arrs - 1:
            # latest data
            feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
            train_valid_merged_admits_latest.append(admit.append(feat_arr_latest))
            # mean and mode
            temp_df = pd.concat(temp_dfs, axis=0)
            feat_arr_mean_mode = temp_df[num_cols].mean().append(temp_df[cat_cols].mode().iloc[0])
            train_valid_merged_admits_mean_mode.append(admit.append(feat_arr_mean_mode))
            # entropy and std
            feat_arr_entropy_std = temp_df[num_cols].std().append(temp_df[cat_cols].apply(lambda x: entropy(x.value_counts()), axis=0))
            train_valid_merged_admits_entropy_std.append(admit.append(feat_arr_entropy_std))
            
train_valid_merged_df_all = pd.DataFrame(train_valid_merged_admits_all)
train_valid_merged_df_latest = pd.DataFrame(train_valid_merged_admits_latest)
train_valid_merged_df_mean_mode = pd.DataFrame(train_valid_merged_admits_mean_mode)
train_valid_merged_df_entropy_std = pd.DataFrame(train_valid_merged_admits_entropy_std)

# test set
test_merged_admits_all = []
test_merged_admits_latest = []
test_merged_admits_mean_mode = []
test_merged_admits_entropy_std = []

for _, admit in test_masked.iterrows():
    num_feat_arrs = len(EHR['feat_dict'][admit['id']])
    temp_dfs = []
    for i in range(num_feat_arrs):
        feat_arr = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
        merged = admit.append(feat_arr)
        merged['obs_id'] = i + 1  
        temp_dfs.append(merged.to_frame().transpose())
        test_merged_admits_all.append(merged)
        if i == num_feat_arrs - 1:
            # latest data
            feat_arr_latest = pd.Series(EHR['feat_dict'][admit['id']][i], index=EHR['feature_cols'])
            test_merged_admits_latest.append(admit.append(feat_arr_latest))
            # mean and mode
            temp_df = pd.concat(temp_dfs, axis=0)
            feat_arr_mean_mode = temp_df[num_cols].mean().append(temp_df[cat_cols].mode().iloc[0])
            test_merged_admits_mean_mode.append(admit.append(feat_arr_mean_mode))
            # entropy and std
            feat_arr_entropy_std = temp_df[num_cols].std().append(temp_df[cat_cols].apply(lambda x: entropy(x.value_counts()), axis=0))
            test_merged_admits_entropy_std.append(admit.append(feat_arr_entropy_std))
            
test_merged_df_all = pd.DataFrame(test_merged_admits_all)
test_merged_df_latest = pd.DataFrame(test_merged_admits_latest)
test_merged_df_mean_mode = pd.DataFrame(test_merged_admits_mean_mode)
test_merged_df_entropy_std = pd.DataFrame(test_merged_admits_entropy_std)

In [194]:
# standardization for numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cat_cols = [EHR['feature_cols'][i] for i in EHR['cat_idxs']]
icd_cols = EHR['icd_cols']
num_cols = list(set(EHR['feature_cols']) - set(cat_cols) - set(icd_cols)) 
num_cols.extend(['stay_len', 'prev_admits']) 

train_valid_merged_df_all[num_cols] = scaler.fit_transform(train_valid_merged_df_all[num_cols])
train_valid_merged_df_mean_mode[num_cols] = scaler.fit_transform(train_valid_merged_df_mean_mode[num_cols])
train_valid_merged_df_latest[num_cols] = scaler.fit_transform(train_valid_merged_df_latest[num_cols])
train_valid_merged_df_entropy_std[num_cols] = scaler.fit_transform(train_valid_merged_df_entropy_std[num_cols])

test_merged_df_all[num_cols] = scaler.fit_transform(test_merged_df_all[num_cols])
test_merged_df_mean_mode[num_cols] = scaler.fit_transform(test_merged_df_mean_mode[num_cols])
test_merged_df_latest[num_cols] = scaler.fit_transform(test_merged_df_latest[num_cols])
test_merged_df_entropy_std[num_cols] = scaler.fit_transform(test_merged_df_entropy_std[num_cols])

In [195]:
# drop features with constant values in the training set
train_valid_merged_df_all.insert(2, 'obs_id', train_valid_merged_df_all.pop('obs_id'))
test_merged_df_all.insert(2, 'obs_id', test_merged_df_all.pop('obs_id'))

feat_cols_all = train_valid_merged_df_all.columns[10:].to_list()
feat_cols_latest = train_valid_merged_df_latest.columns[9:].to_list()
feat_cols_mean_mode = train_valid_merged_df_mean_mode.columns[9:].to_list()
feat_cols_entropy_std = train_valid_merged_df_entropy_std.columns[9:].to_list()

const_cols_all = [col for col in feat_cols_all if train_valid_merged_df_all[col].nunique() <= 1]
const_cols_latest = [col for col in feat_cols_latest if train_valid_merged_df_latest[col].nunique() <= 1]
const_cols_mean_mode = [col for col in feat_cols_mean_mode if train_valid_merged_df_mean_mode[col].nunique() <= 1]
const_cols_entropy_std = [col for col in feat_cols_entropy_std if train_valid_merged_df_entropy_std[col].nunique() <= 1]
print(const_cols_all)
print(const_cols_latest)
print(const_cols_mean_mode)
print(const_cols_entropy_std)

train_valid_merged_df_all = train_valid_merged_df_all.drop(columns=const_cols_all, axis=1)
test_merged_df_all = test_merged_df_all.drop(columns=const_cols_all, axis=1)

train_valid_merged_df_latest = train_valid_merged_df_latest.drop(columns=const_cols_latest, axis=1)
test_merged_df_latest = test_merged_df_latest.drop(columns=const_cols_latest, axis=1)

train_valid_merged_df_mean_mode = train_valid_merged_df_mean_mode.drop(columns=const_cols_mean_mode, axis=1)
test_merged_df_mean_mode = test_merged_df_mean_mode.drop(columns=const_cols_mean_mode, axis=1)

train_valid_merged_df_entropy_std = train_valid_merged_df_entropy_std.drop(columns=const_cols_entropy_std, axis=1)
test_merged_df_entropy_std = test_merged_df_entropy_std.drop(columns=const_cols_entropy_std, axis=1)

print(train_valid_merged_df_all.shape)
print(train_valid_merged_df_latest.shape)
print(train_valid_merged_df_mean_mode.shape)
print(train_valid_merged_df_entropy_std.shape)

['A20-A28', 'J00-J06']
['A20-A28', 'J00-J06', 'Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'pH Urine', 'Basophils Joint Fluid', 'Basophils Ascites', 'Lymphocytes Joint Fluid']
['Basophils Other Body Fluid', 'Basophils Pleural', 'Eosinophils Joint Fluid', 'Eosinophils Ascites', 'pH Urine', 'Lymphocytes Ascites', 'Eosinophils Other Body Fluid', 'Basophils Joint Fluid', 'H Blood', 'Basophils Ascites', 'Lymphocytes Other Body Fluid', 'Monocytes Ascites', 'Lymphocytes Joint Fluid', 'A20-A28', 'J00-J06']
['MUSCLE RELAXANTS', 'CARDIOVASCULAR', 'ANTINEOPLASTICS', 'ANTIPARKINSON DRUGS', 'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS', 'PRE-NATAL VITAMINS', 'ANTIPARASITICS', 'SKIN PREPS', 'CONTRACEPTIVES', 'THYROID PREPS', 'DIAGNOSTIC', 'ANTIVIRALS', 'ANTIDOTES', 'ANTIINFECTIVES/MISCELLANEOUS', 'CNS DRUGS', 'UNCLASSIFIED DRUG PRODUCTS', 'age', 'gender', 'ethnicity', 'Y90-Y99', 'G30-G32', 'O85-O92', 'C60-C63', 'F40-F48', 'M80-M85', 'R00-R09', 'J90-J94', 'A00-

In [196]:
# make copies
#train_valid_merged_df_all_copy = train_valid_merged_df_all.copy()
#train_valid_merged_df_median_copy = train_valid_merged_df_median.copy()
#train_valid_merged_df_latest_copy = train_valid_merged_df_latest.copy()

In [197]:
# drop info columns
#train_valid_merged_df_all = train_valid_merged_df_all.drop(train_valid_merged_df_all.columns[0:9], axis=1)
#train_valid_merged_df_latest = train_valid_merged_df_latest.drop(train_valid_merged_df_latest.columns[0:8], axis=1)
#train_valid_merged_df_median = train_valid_merged_df_median.drop(train_valid_merged_df_median.columns[0:8], axis=1)

In [198]:
# feature selection with correlation test
# 1. drop features with correlation (with response) lower than 0.1 (5/10 fold correlation test)
# 2. if multiple features have correlation > 0.7 between each other, only keep the one with the highest correlation with response

#train_merged_df_all
#STEP1
# feature selection
# 1. drop features with correlation (with response) lower than 0.1 (5/10 fold correlation test)
# 2. if multiple features have correlation > 0.7 between each other, only keep the one with the highest correlation with response

#train_merged_df_all
#STEP1

#k = 10
#cv = KFold(n_splits=k, shuffle=True)
#selected_features = []
#for train_index, _ in cv.split(train_merged_df_all):
#    fold_data = train_merged_df_all.iloc[train_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#compute average correlation
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#drop variables with correlation with response lower than 0.01
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_selected_df_all = train_merged_df_all.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_selected_df_all.corr()
#cols = corr_matrix.columns
#drop variables with correlation with correlation between each other>0.7 except the highest one
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_selected_df_all = train_selected_df_all.drop(cols[j], axis=1)
#            else:
#                train_selected_df_all = train_selected_df_all.drop(cols[i], axis=1)
#                break
#print(train_selected_df_all.shape)

#train_merged_df_latest
#STEP1
#k = 10
#cv = KFold(n_splits=k, shuffle=True)
#selected_features = []
#for train_index, _ in cv.split(train_merged_df_latest):
#    fold_data = train_merged_df_latest.iloc[train_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_selected_df_latest = train_merged_df_latest.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_selected_df_latest.corr()
#cols = corr_matrix.columns
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_selected_df_latest = train_selected_df_latest.drop(cols[j], axis=1)
#            else:
#                train_selected_df_latest = train_selected_df_latest.drop(cols[i], axis=1)
#                break
#print(train_selected_df_latest.shape)

#train_merged_df_median
#STEP1
#k = 10
#cv = KFold(n_splits=k, shuffle=True)
#selected_features = []
#for train_index, _ in cv.split(train_merged_df_median):
#    fold_data = train_merged_df_median.iloc[train_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_selected_df_median = train_merged_df_median.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_selected_df_median.corr()
#cols = corr_matrix.columns
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_selected_df_median = train_selected_df_median.drop(cols[j], axis=1)
#            else:
#                train_selected_df_median = train_selected_df_median.drop(cols[i], axis=1)
#                break
#print(train_selected_df_median.shape)

In [199]:
#train_valid_merged_df_all
#STEP1
#k = 10
#cv = KFold(n_splits=k, shuffle=True)
#selected_features = []
#for train_valid_index, _ in cv.split(train_valid_merged_df_all):
#    fold_data = train_valid_merged_df_all.iloc[train_valid_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_valid_selected_df_all = train_valid_merged_df_all.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_valid_selected_df_all.corr()
#cols = corr_matrix.columns
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_valid_selected_df_all = train_valid_selected_df_all.drop(cols[j], axis=1)
#            else:
#                train_valid_selected_df_all = train_valid_selected_df_all.drop(cols[i], axis=1)
#                break
#print(train_valid_selected_df_all.shape)

#train_valid_merged_df_latest
#STEP1
#k = 10
#cv = KFold(n_splits=k)
#selected_features = []
#for train_valid_index, _ in cv.split(train_valid_merged_df_latest):
#    fold_data = train_valid_merged_df_latest.iloc[train_valid_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_valid_selected_df_latest = train_valid_merged_df_latest.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_valid_selected_df_latest.corr()
#cols = corr_matrix.columns
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_valid_selected_df_latest = train_valid_selected_df_latest.drop(cols[j], axis=1)
#            else:
#                train_valid_selected_df_latest = train_valid_selected_df_latest.drop(cols[i], axis=1)
#                break
#print(train_valid_selected_df_latest.shape)

#train_valid_merged_df_median
#STEP1
#k = 10
#cv = KFold(n_splits=k)
#selected_features = []
#for train_valid_index, _ in cv.split(train_valid_merged_df_median):
#    fold_data = train_valid_merged_df_median.iloc[train_valid_index]
#    numeric_data = fold_data.select_dtypes(include=np.number) 
#    correlations = numeric_data.corrwith(fold_data['readmitted_within_30days'])
#    selected_features.append(correlations)
#selected_features_df = pd.concat(selected_features, axis=1)
#average_correlation = selected_features_df.mean(axis=1)
#selected_columns = average_correlation[average_correlation < 0.001].index
#train_valid_selected_df_median = train_valid_merged_df_median.drop(selected_columns, axis=1)
#STEP2
#corr_matrix = train_valid_selected_df_median.corr()
#cols = corr_matrix.columns
#for i in range(len(cols)):
#    for j in range(i+1, len(cols)):
#        if abs(corr_matrix.iloc[i,j]) > 0.7:
#            if abs(corr_matrix.iloc[i,-1]) > abs(corr_matrix.iloc[j,-1]):
#                train_valid_selected_df_median = train_valid_selected_df_median.drop(cols[j], axis=1)
#            else:
#                train_valid_selected_df_median = train_valid_selected_df_median.drop(cols[i], axis=1)
#                break
#print(train_valid_selected_df_median.shape)

In [200]:
# add suffix
train_valid_merged_df_entropy_std.columns = list(train_valid_merged_df_entropy_std.columns[:9]) + [str(col) + '_entropy_std' for col in train_valid_merged_df_entropy_std.columns[9:]]
test_merged_df_entropy_std.columns = list(test_merged_df_entropy_std.columns[:8]) + [str(col) + '_entropy_std' for col in test_merged_df_entropy_std.columns[8:]]

train_valid_merged_df_mean_mode.columns = list(train_valid_merged_df_mean_mode.columns[:9]) + [str(col) + '_mean_mode' for col in train_valid_merged_df_mean_mode.columns[9:]]
test_merged_df_mean_mode.columns = list(test_merged_df_mean_mode.columns[:8]) + [str(col) + '_mean_mode' for col in test_merged_df_mean_mode.columns[8:]]

train_valid_merged_df_latest.columns = list(train_valid_merged_df_latest.columns[:9]) + [str(col) + '_latest' for col in train_valid_merged_df_latest.columns[9:]]
test_merged_df_latest.columns = list(test_merged_df_latest.columns[:8]) + [str(col) + '_latest' for col in test_merged_df_latest.columns[8:]]

train_valid_merged_df_all.columns = list(train_valid_merged_df_all.columns[:9]) + [str(col) + '_all' for col in train_valid_merged_df_all.columns[9:]]
test_merged_df_all.columns = list(test_merged_df_all.columns[:8]) + [str(col) + '_all' for col in test_merged_df_all.columns[8:]]

In [201]:
# write csv files
# train set
train_valid_merged_df_latest.to_csv('train/train_valid_latest.csv', index=False)
train_valid_merged_df_mean_mode.to_csv('train/train_valid_mean_mode.csv', index=False)
train_valid_merged_df_all.to_csv('train/train_valid_all.csv', index=False)
train_valid_merged_df_entropy_std.to_csv('train/train_valid_entropy_std.csv', index=False)

with zipfile.ZipFile('train/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('train/train_valid_all.csv')
os.remove('train/train_valid_all.csv')

# test set
test_merged_df_latest.to_csv('test/test_latest.csv', index=False)
test_merged_df_mean_mode.to_csv('test/test_mean_mode.csv', index=False)
test_merged_df_entropy_std.to_csv('test/test_entropy_std.csv', index=False)
test_merged_df_all.to_csv('test/test_all.csv', index=False)
with zipfile.ZipFile('test/sequence_data.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('test/test_all.csv')
os.remove('test/test_all.csv')